Second developement notebook of code for the card game skyjo.
First step is to implement and test one mode which uses xgbregressor results. 


In [1]:
import numpy as np
import random as random
#using simpleguitk for display, is not needed for computer game
import simpleguitk as simplegui
import time
#new ones
import pandas as pd
from xgboost import XGBRegressor

Below the functions are copied it are the same as in skyjo_exploring.ipynb if not noted otherwise.

For the card class I add the method numeric, very similar to for Pile class and player class.

In [2]:
#cards of the game
class Card:
    #initiate cards
    def __init__(self, number):
        #dictionary for defining colors
        dict_col={-2:'blue',-1:'blue',0:'cyan',1:'green',2:'green',3:'green',4:'green',5:'yellow',6:'yellow',7:'yellow',8:'yellow',9:'red',10:'red',11:'red',12:'red'}
        self.number = number        
        self.color = dict_col[number]
        #open or closed, default is closed
        self.open =False
        #default position is None
        self.position=[None,None]  
    #print output    
    def __str__(self):
        if self.open==False:
            return "Card has Value "+str(self.number)+" and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Card has Value "+str(self.number)+" and is open."+ \
                " Its position is at "+str(self.position)
    #set card to a state
    def set_state(self,state):
        self.open=state
    #set card to number, mainly for testing    
    def set_number(self,number):
        #dictionary twice defined that class does not need outside information
        dict_col={-2:'blue',-1:'blue',0:'cyan',1:'green',2:'green',3:'green',4:'green',5:'yellow',6:'yellow',7:'yellow',8:'yellow',9:'red',10:'red',11:'red',12:'red'}
        self.number=number
        self.color = dict_col[number]
    #set position for display  
    def set_position(self,posinput):
        self.position=posinput
    #draw on GUi    
    def draw(self,canvas):
        card_size=[70,50]
        corners=[[self.position],[self.position[0]+card_size[0],self.position[1]],[self.position[0]+card_size[0],self.position[1]+card_size[1]],[self.position[0],self.position[1]+card_size[1]]]
        if self.open==False:
            #closed black
            canvas.draw_polygon(corners,1,'Black','Black')
        else:
            #open colored number on gray
            canvas.draw_polygon(corners,1,'Black','Light Gray')
            centerb=list(self.position)
            centerb[1]+=58
            centerb[0]+=5
            canvas.draw_text(self.number,centerb,50,self.color)
    #get value (not getting number) when card not open before a turn is finished        
    def get_numeric(self):
        if self.open==1:
            value=self.number
        else:
            value=20
        return value            

To the pile and player class I add the method copy card, for the use when simulating (exploring the draw options) that way the pile/player is not changed then.

I change the numeric output of the pile class now a histogram of the values is returned. That makes the output smaller

In [121]:
#pile of cards can be open or closed, only top most accessable
class Pile:
    #create parameter: string of mode, where to take cards for it, can be None
    def __init__(self,mode,take_from):
        #create new close pile, shuffles it also 
        if mode=='create_closed':
            #all closed in this mode
            self.open=False
            #postion in gui
            self.position=[100,10]
            #number of cards per type
            dict_num={-2:5,-1:10,0:15,1:10,2:10,3:10,4:10,5:10,6:10,7:10,8:10,9:10,10:10,11:10,12:10}
            #create empty list
            self.list_cards=[]
            #add as many cards as exist of each type
            for i in range(15):
                for j in range(dict_num[i-2]):
                    #create card
                    cardin=Card(i-2)
                    #use position of Pile
                    cardin.set_position(self.position)
                    #and state of Pile
                    cardin.set_state(self.open)
                    #append
                    self.list_cards.append(cardin)    
                    #print content 
                    self.list_cards
            #shuffle cards        
            random.shuffle(self.list_cards)     
        #create open pile from other    starts with just one card
        if mode=='create_open': 
            #postion in gui
            self.position=[300,10]   
            #is open 
            self.open=True
            self.list_cards=[]
            self.list_cards.append(take_from.list_cards.pop(-1))
            #set card position properties to Pile properties
            self.list_cards[0].set_state(self.open)
            self.list_cards[0].set_position(self.position)            
    def __str__(self): 
        #print Pile properties
        if self.open==False:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is open."+ \
                " Its position is at "+str(self.position)
    #giving a card default is top most (last) cards
    def give_card(self,position=-1):
        card=self.list_cards.pop(position)
        return card
    #copy card for simulation
    def copy_card(self,position=-1):
        card=self.list_cards[position]
        return card
    #get card, added at as the last (top most ) always
    def get_card(self,card):
        self.list_cards.append(card)
        #set card position and state to position ans state of pile
        self.list_cards[-1].set_position(self.position)
        self.list_cards[-1].set_state(self.open)
    #recreate pile if nearly empty. parameters: self=pile to be recreated), other pile to be used        
    def refill(self,p_open):
        #refill if there are less than 3 cards
        if len(self.list_cards)<3:
            while len(p_open.list_cards)>1:
                #card closest to the bottom is taken until there is only 1 card
                #cannot use other method in line below, then the top most is taken many times
                card_a=p_open.list_cards.pop(0)
                self.get_card(card_a)
            #shuffle again
            random.shuffle(self.list_cards)
            #return other pile, self in anyway right
            return p_open
    #gets output for machine learning input     
    #gets numpy formated 1D array of 150 length (could be 147 in principle, 3 cards are always elsewhere), value is number when open, closed 20, non existend 30   
    #starts with end because the is top most, most important card
    #all cards besides the first one are ordered by value since the order doe snot matter for 
    #them only the first one can be gottten back
    def get_numeric(self):
        ar=np.zeros((16),int)
        #first is top most card because it can be accesed it is separated
        #only on open cards as savegard
        if self.open==1:
            ar[0]=self.list_cards[-1].number
        #if other cards histogram of the rest, -2 to 12 in order
        if len(self.list_cards)>1:
            for i in range(len(self.list_cards)-1):
                if self.open==1:
                    ar[self.list_cards[i].number+3]+=1
        
        return ar         

Here I test the new numeric output for piles. 

In [124]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
card1=pile_closed.give_card()
print(card1)
card2=pile_closed.give_card()
print(card2)
ar1=pile_closed.get_numeric()
print(ar1)
ar2=pile_open.get_numeric()
print(ar2)
pile_open.get_card(card1)
pile_open.get_card(card2)
ar2b=pile_open.get_numeric()
print(ar2b)

Card has Value 7 and is closed. Its position is at [100, 10]
Card has Value 8 and is closed. Its position is at [100, 10]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[8 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]


In [125]:
#A Card Position can be empty. I use two lists here for the essential properties, one to check for empty and
#one which has stored the card. 
class Player:
    #parameters: string name, mode (human, computer),  level (subkind for mode, more options for computer then human) 
    def __init__(self,name,modes,level,take_from=None):
        #grid positions only for display the game logic uses the list position
        self.positionx=[0,70,140,210,0,70,140,210,0,70,140,210]
        self.positiony=[0,0,0,0,50,50,50,50,100,100,100,100]
        #whether cards exist
        self.exist=[]
        #which cards is there
        self.list_cards=[]
        #name of player
        self.name=name
        #check mode computer (different levels)/human
        self.mode=modes
        #level how random much randomness is there is     
        self.level=level 
        #if take_from is None, only player properties are defined but no cards given
        if take_from!=None:
            for i in range(12):
                #add card which exist
                self.exist.append(1)
                #from pile
                self.list_cards.append(take_from.list_cards.pop(-1))
                #set state to closed
                self.list_cards[-1].set_state(False)
                #set position of card to player grid
                pos=list([self.positionx[i],self.positiony[i]])
                self.list_cards[-1].set_position(pos) 
            #opening 2 cards, currently only open randomly is implemented
            if level<10:
                rang=list(range(12))
                random.shuffle(rang)
                self.list_cards[rang[0]].set_state(True)
                self.list_cards[rang[1]].set_state(True)
        #option for player to do that interactive to be added   
    #for further round in a game only recreate, rest is the same as above in in __init__
    def restart(self,take_from):
        self.exist=[]
        self.list_cards=[]   
        for i in range(12):
            self.exist.append(1)
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[-1].set_state(False)
            pos=list([self.positionx[i],self.positiony[i]])
            self.list_cards[-1].set_position(pos) 
        if self.level<10:
            rang=list(range(12))
            random.shuffle(rang)
            self.list_cards[rang[0]].set_state(True)
            self.list_cards[rang[1]].set_state(True) 
    #printing basic player properties        
    def __str__(self):
        return "Player "+self.name+" is a "+self.mode+" in level "+str(self.level)+"."
    #give a card, choosen by index, vanishes: is set to None and not existing
    def give_card(self,i):
        card=self.list_cards[i]
        self.exist[i]=0
        self.list_cards[i]=None
        return card
    #copy card for simulation
    def copy_card(self,i):
        card=self.list_cards[i]
        return card    
    #Get a card (parameters: card and index), is always open by design
    def get_card(self,card,i):
        #change ccard
        self.list_cards[i]=card
        #set to existing
        self.exist[i]=1
        #get position of the adress
        pos=list([self.positionx[i],self.positiony[i]])
        #use it
        self.list_cards[i].set_position(pos)
        self.list_cards[i].set_state(True)
    #set_value method only  used in testing    
    #paremeters, index of card in list, number, state     
    def set_value(self,i,number,state):
        #set to existing, open state and desired value
        self.exist[i]=1  
        self.list_cards[i].set_state(True)     
        self.list_cards[i].set_number(number)        
    #method to get list of all closed cards, optional print of how many there are    
    def get_all_closed(self,silent=True):
        #create empty list, append
        closed=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==False:
                    closed.append(j)
        if silent==False:      
            print(self.name+" has "+str(len(closed))+" closed cards")            
        return closed     
    #method to get list of all open cards, optional print of how many there are  
    #same structure as above
    def get_all_open(self,silent=True):   
        open2=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==True:
                    open2.append(j)
        if silent==False:
            print(self.name+" has "+str(len(open2))+" open cards")            
        return open2 
    #method to get list of all existing cards, optional print of how many there are  
    #same structure as above    
    def get_all_cards(self,silent=True):   
        existing=[]
        for j in range(12):
            if self.exist[j]==1:
                existing.append(j)
        if silent==False:
            print(self.name+" has "+str(len(existing))+" cards")            
        return existing    
    #get score at the end of a round, adidng up all number values of existing cards
    def get_score(self):   
        score=0
        for j in range(12):
            if self.exist[j]==1:
                score+=self.list_cards[j].number         
        return score 
    #when 3 open cards in column have the same value, they vanish 
    #check whether cards vanish
    def check_vanish_cards(self):
        #-1 is default when no vanished
        vanish=-1
        #checking the four options
        for j in range(4):
            #3 open of same number
            if self.exist[0+j]==True and self.exist[4+j]==True and self.exist[8+j]==True: 
                if self.list_cards[0+j].open==True and self.list_cards[4+j].open==True and self.list_cards[8+j].open==True and self.list_cards[0+j].number==self.list_cards[4+j].number and self.list_cards[0+j].number==self.list_cards[8+j].number:
                    #is yes index is returned by design there can only one row be added in each turn, thus it is fine
                    #that the methiod cannot handle several
                    vanish=j
        return vanish  
    #method which return vanished when previous method marks them (gets more than -1)
    def vanish_cards(self,row):
        card1=self.list_cards[0+row]
        card2=self.list_cards[4+row]
        card3=self.list_cards[8+row]
        self.exist[0+row]=0
        self.exist[4+row]=0
        self.exist[8+row]=0
        self.list_cards[0+row]=None
        self.list_cards[4+row]=None
        self.list_cards[8+row]=None
        return card1, card2, card3 
    #gets numeric output for machine learning, same principle than for piles
    #gets numpy formated 1D array of 12 length, values is number when open, closed 20, non existedn 30   
    def get_numeric(self):
        ar=np.zeros((12),int)
        for i in range(12):
            if self.exist[i]==0:
                ar[i]=30
            else:
                if self.list_cards[i].open==1:
                    ar[i]=self.list_cards[i].number
                else:
                    ar[i]=20
        return ar   

In [126]:
#test card copying
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',0,pile_closed)
beta=Player("beta",'computer',0,pile_closed) 
players=[alpha,beta]
card1=pile_closed.copy_card()
#does not change state needs to be applied on it sometimes
print(card1)
card2=alpha.copy_card(2)
print(card2)
print(card2.get_numeric())
print(card1.get_numeric())

Card has Value 0 and is closed. Its position is at [100, 10]
Card has Value 2 and is closed. Its position is at [140, 0]
20
20


In [127]:
#function which dtermines who storts
#parameters: whether is first round of game, player_list, which player ended the last round, 
#,silent whether the result is printed on screen
def who_starts(first,players,last_ender,silent=True):
    #if first round of a game, player with largest sum of open values starts
    if first==True:
        #counter list for all players
        counters=[]
        for i in range(len(players)):
            #counting for each player
            counter=0
            #get all open cards #get there indixes
            open_cards=players[i].get_all_open(silent=True)
            for j in range(len(open_cards)):
                #get values of these cards
                counter+=players[i].list_cards[open_cards[j]].number
            counters.append(counter)            
        #get maximum
        maxc=max(counters)
        #get index of it, not important whoch chossen when there are several
        starter=players[counters.index(maxc)]
        if silent==False:
            print("Player "+str(starter.name)+" starts")        
        return starter 
    #otherwise the ender of the previous round starts
    else:
        if silent==False:
            print("Player "+str(last_ender.name)+" starts") 
        return last_ender                    

Next I build a function to make a turn, the basic unit of the game skyjo. 
This function has many mode, first I start with some computer option. Level 0 of the computer is just random and will be used to generate the data for the machine learning to be trained from. 
Level -1, -2 -3 of the computer use non randomly the different possible actions. That is mainly use to check whether the actions work. 

Optionally following outout is collected  on the turn level for the machine learning. It consists of the state before the action, plus the then done actions. 
Collected are the player index [1 variable], the player state (12 variables), other player state [12*(n-1) variables], open pile state (16), closed pile length (1). The actions are boleans for take_open [1 variables] and discarding cards [1 variable]. The value of the card closed pile (set to dummy if closed is not used). Finally the index, the value and state of the choosen player card (combined) [2 variables]. That are in total 23+12*N(players)

Now I implement the first XGBoost results.  For it I first take out the actual action from the turn into its own function. In addition to just executing one by turn selected option, I also add an option simulated in which all options are explored and then the model is applied on. 

In [74]:
level1_2players_columns=np.loadtxt("xgb_model1_column1.txt")
level1_2players_model = XGBRegressor()
level1_2players_model.load_model("xgb_model1.json")
print(sum(level1_2players_columns))
#dictionary to get models and columns
player_2models={1:level1_2players_model}
player_2columns={1:level1_2players_columns}
act_array=np.array([[1,0],[0,1],[0,0]])

31.0


Helper function which converts a list of of euqual length list to a numpy array. 

In [75]:
#arguments lists, default is an integer array since here all are integers
def lists_to_numpy(the_list,int2=True):
    if int2==True:
        array=np.zeros((len(the_list),len(the_list[0])),int)
    else:
        array=np.zeros((len(the_list),len(the_list[0])))
    for i in range(len(the_list)):
        array[i]=the_list[i]
    #transpose it to consistent with previous way below    
    return array.T                   

In [90]:
list_a=[[0,1,3,4],[2,2,5,6],[3,4,7,8]]
res=lists_to_numpy(list_a)
print(res.shape,type(res),res)


(4, 3) <class 'numpy.ndarray'> [[0 2 3]
 [1 2 4]
 [3 5 7]
 [4 6 8]]


Thought about data structure. There are three kinds of actions, for normal turns just one is done, but for the exploring of options all need to be explored. Can be built easiest when it is always a loop over it  and partly several actions are done and other times just one. 

In the past the collected numbers used closed numbers during the turn, that is now changed. Means that more new games need to be run. 

In [145]:
#defines actions they are either done or just simulated(=explored to see which is best)
#parameters, plater, all players card, closed pile, discarded pile, take_open, discard
#optionla non silent, simulated, and whether a certain card (index) should be used 
#now always output collect and propagated easier this way
def actions(player,players,pile,discarded,take_open, discard,silent=True,simulated=False,card=0):
    #now numeric output always collected (but necessarily passed) since needed for simulating level 1
    num=np.zeros((23+12*len(players)),int)
    #which players turn it is
    for i in range(len(players)):
        if players[i]==player:
            num[0]=i
    #discard and closed pile properties in data collection       
    num[1:17]=discarded.get_numeric()
    num[17]=len(pile.list_cards)  
    #cards of current players turn
    num[18:30]=player.get_numeric()
    #get of other players
    #counting the player to insert at right place
    player_counter=0
    for i in range(len(players)):
        #exclduing current player
        if players[i]!=player:
            player_counter+=1
            other_player=players[i]
            #get numeric of other players
            num[18+player_counter*12:30+player_counter*12]=other_player.get_numeric()    
    #numeric data collectd in lists (length unknown) in case of simulated  
    if simulated==True:
        data=[]
    #actions array for simulated 3 action options 3 2 array
    act_array=np.array([[1,0],[0,1],[0,0]])
    #iterate over options alwys  how many depends on mode
    for j in range(3):
        if j==0 and ((simulated==False and act_array[j,0]==take_open and act_array[j,1]==discard) or simulated==True):
            if silent==False:
                print(f"case {j} take_open={act_array[j,0]} discard={act_array[j,1]}")            
            num2=np.copy(num)
            num2[30+player_counter*12]=act_array[j,0]
            num2[31+player_counter*12]=act_array[j,1]
            if act_array[j,0]==1 and act_array[j,1]==0:
                if silent==False and simulated==False:
                    print("take from open swaping with own")
            #use method to get existing cards
            existing=player.get_all_cards()
            #select from them
            #randomly for implemented levels 0
            if player.mode=='computer':
                if player.level>=-3 and player.level<=0:
                    #index of selected card
                    selected=[random.choice(existing)]
                #for simulated this one currently one implemented    
                if player.level==1 and simulated==True:
                    selected=existing.copy()
            #get selected cards
            for i in range(len(selected)):
                if simulated==False:
                    card_a=player.give_card(selected[i])
                else:    
                    card_a=player.copy_card(selected[i])                
                #dummy for closed pile value that entry is not empty
                num3=np.copy(num2)
                num3[32+player_counter*12]=30
                # get selected card properties
                num3[33+player_counter*12]=selected[i]
                num3[34+player_counter*12]=card_a.get_numeric()
                #exchanging cards with open pile
                if simulated==False:
                    card_b=pile_open.give_card()
                    pile_open.get_card(card_a)
                    player.get_card(card_b,selected[i])
                else:
                    #collect data
                    data.append(num3)
            if silent==False and simulated==False:            
                print("player "+player.name+" has now "+str(player.list_cards[selected[i]].number))
                print("open Pile has now on "+str(pile_open.list_cards[-1].number))                
        #other options take from close pile, discard it open closed own 
        if j==1 and ((simulated==False and act_array[j,0]==take_open and act_array[j,1]==discard) or simulated==True):         
            if silent==False:
                print(f"case {j} take_open={act_array[j,0]} discard={act_array[j,1]}")   
            num2=np.copy(num)
            num2[30+player_counter*12]=act_array[j,0]
            num2[31+player_counter*12]=act_array[j,1]
            if silent==False and simulated==False:                
                print("take from closed discards it")
            #put card on open pile from closed
            if simulated==False:
                card_a=pile_closed.give_card()
            if simulated==True:
                card_a=pile_closed.copy_card()       
            #numeric output collection
            num2[32+player_counter*12]=card_a.number 
            if simulated==False:
                pile_open.get_card(card_a)
            #get lists of all closed player cards
            closed=player.get_all_closed()
            #computer options
            if player.mode=='computer':  
                #all implemented are random               
                if player.level<=0 and player.level>=-3:
                    selected=[random.choice(closed)]  
                if player.level==1 and simulated==True:     
                    selected=closed.copy()
            #that random choice is passed to collected data
            if simulated==True:
                print(f"length is {len(selected)}")
            for i in range(len(selected)):
                if silent==False and simulated==False:
                    print("Player opens "+str(player.list_cards[selected[i]].number))   
                num3=num2.copy()   
                num3[33+player_counter*12]=selected[i]
                num3[34+player_counter*12]=player.list_cards[selected[i]].get_numeric()
                #selected card is set to open
                if simulated==False:          
                    player.list_cards[selected[i]].set_state(True)     
                else:  
                    # add numeric to data
                    data.append(num3)       
            if silent==False and simulated==False:                
                print("open Pile has now on top "+str(pile_open.list_cards[-1].number))     
        #next (last) action option swap closed pile card with own
        if j==2 and ((simulated==False and act_array[j,0]==take_open and act_array[j,1]==discard) or simulated==True):              
            if silent==False:
                print(f"case {j} take_open={act_array[j,0]} discard={act_array[j,1]}")   
            num2=np.copy(num)
            num2[30+player_counter*12]=act_array[j,0]
            num2[31+player_counter*12]=act_array[j,1]
            if silent==False and simulated==False:
                print("take from closed swaps with own")
            #get all existing cards
            existing=player.get_all_cards()
            if player.mode=='computer':
                if player.level>=-3 and player.level<=0:
                    #index of selected card
                    selected=[random.choice(existing)]
                #for simulated this one currently one implemented    
                if player.level==1 and simulated==True:
                    selected=existing.copy()
            #get selected cards
            for i in range(len(selected)):
                num3=num2.copy() 
                if simulated==False:
                    card_a=player.give_card(selected[i])
                    card_b=pile_closed.give_card() 
                else:    
                    card_a=player.copy_card(selected[i])   
                    card_b=pile_closed.copy_card() 
                #dummy for closed pile value that entry is not empty
                num3[32+player_counter*12]=card_b.number
                # get selected card properties
                num3[33+player_counter*12]=selected[i]
                num3[34+player_counter*12]=card_a.get_numeric()
                #exchanging cards with open pile
                if simulated==False:
                    pile_open.get_card(card_a)
                    player.get_card(card_b,selected[i])
                else:
                    #collect data
                    data.append(num3)   
            if silent==False and simulated==False:                
                    print("player "+player.name+" has now "+str(player.list_cards[selected[i]].number))                
                    print("open Pile has now in top "+str(pile_open.list_cards[-1].number))    
        if silent==False and simulated==True:
            print(f"number of simulations collected until now: {len(data)}")
    #full aray outut when simulated         
    if simulated==True:
        return lists_to_numpy(data)
    #only 1D array when not
    if simulated==False:
        return num3    

Now I test how it works in the standard modes and in simulated.
The application of other modes is not implemented now because i discovered that the wrong dat collected previously. 

In [144]:
#testing the past modes
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',1,pile_closed)
beta=Player("beta",'computer',1,pile_closed) 
players=[alpha,beta]
#always same player in the testing below
res=actions(alpha,players,pile_closed,pile_open,True, False, silent=False,simulated=True)
print(res[:,3]-res[:,5])
print(res[:,13]-res[:,15])
print(res[:,23]-res[:,25])
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',0,pile_closed)
beta=Player("beta",'computer',0,pile_closed) 
players=[alpha,beta]
res2=actions(beta,players,pile_closed,pile_open,True, False, silent=False,simulated=False)
print(res2)
res3=actions(beta,players,pile_closed,pile_open,True, False, silent=True,simulated=False)
#print(res3)

case 0 take_open=1 discard=0
number of simulations collected until now: 12
case 1 take_open=0 discard=1
length is 10
number of simulations collected until now: 22
case 2 take_open=0 discard=0
number of simulations collected until now: 34
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -3  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0]
case 0 take_open=1 discard=0
take from open swaping with own
player beta has now 5
open Pile has now on 0
[  1   5   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 125
  20  20  20  20  20   9  20   1  20  20  20  20  20  20  20  20   8  20
  20  11  20  20  20  20   1   0  30   4  20]
[  1   0   0   0   0   0   0   0

I now rewrite turn since it now uses the action function and its output. 

It is now much shorter, it just set the value of take_open and discard, and does the after action controlls for vanishing cards, refill and wehther the game is finished. 

Now it is added to the numeric output whether the vanishing card event happened. (Not always predictable but still done for now anyway not used in the first round of machine learning). 


In [160]:
#parameters: current player, all players (only needed for numeric output collection and for choosing startegry in some levels, closed_pile, discarded_pile, 
#Currently implemented mode with levels 0, -1, -2, -3
def turn(player,players,pile,discarded,silent=True,output=False):
    #global in_play parameter to check whether the game is over for one player
    global in_play  #, player_2models, player_2columns for later   
    #set take_open and discard
    if player.mode=='computer':
        #in level 0 random 50% choice of action
        if player.level==0:
            r_number1=random.random()
            if r_number1<0.5:
                take_open=False
            else:    
                take_open=True
            #discard random if one case
            if take_open==False:
                r_number2=random.random()                
                if r_number2<0.5:
                    discard=False
                else:    
                    discard=True   
            else:
                #set to false for take_open is true
                discard=False
        #later further modes level 1 or other with similar approaches with 2 players        
        #if player.level>0 and len(players)==2:
            #model=player_2models[player.level]
            #columns=player_2columns[player.level]               
        #level -1/-2/-3 have assigned choices        
        if player.level==-1 or player.level==-2:
            take_open=False
        if player.level==-3:
            take_open=True
        if player.level==-1 or player.level==-3:
                discard=False
        if player.level==-2:
                discard=True            
    #now action function
    if silent==False:
        print("player "+player.name+" turn")
    num=actions(player,players,pile_closed,pile_open,take_open,discard,silent=silent)   
    #check for vanishing cards
    van=player.check_vanish_cards()
    #marker for card vanishing
    card_needs_to_vanish=False
    if van>-1:
        card_needs_to_vanish=True
        #cards are taken from player
        card1,card2,card3=player.vanish_cards(van)
        #given to open pile
        pile_open.get_card(card1)
        pile_open.get_card(card2)
        pile_open.get_card(card3)
        if silent==False:        
            print("3 "+str(card1.number)+" vanish from Player")
    #refill closed pile if needed 
    pile_closed.refill(pile_open)
    #check whether there are still closed cards 
    closed=player.get_all_closed()
    #if not play ends for this player and marker is set to not in_play
    if len(closed)==0:
        in_play=False 
        if silent==False:        
            print("player "+player.name+" opened all cards")
    if output==False:
        #with no output just collected the input again (needed?)
        return player, players, pile, discarded  
    else:
        #otherwise collect output and also marker on whether cards were vanished 
        #(useful to know whether enough data was collected that the machine learning can consider it)
        #combine num, card to vanish
        num2=np.zeros((len(num)+1))
        num2[:len(num)]=num
        num2[len(num)]=card_needs_to_vanish
        return player, players, pile, discarded, num2

Now I test wehther it still works.

In [161]:
#first level (computer mode) 0
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',0,pile_closed)
beta=Player("beta",'computer',0,pile_closed) 
players=[alpha,beta]
#always same player in the testing below
#all not silent and output collection
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("full numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
print(num.shape)

player alpha turn
case 2 take_open=0 discard=0
take from closed swaps with own
player alpha has now 9
open Pile has now in top 9
number of closed cards of player alpha is 9
full numeric output is
[  0.   3.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 125.  20.   5.  10.  20.  20.  20.  20.  20.  20.  20.
  20.  20.  20.  20.  20.  20.  20.  -1.  20.   0.  20.  20.  20.  20.
   0.   0.   9.   0.  20.   0.]
player alpha numeric output is
[20.  5. 10. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
player beta numeric output is
[20. 20. 20. 20. 20. -1. 20.  0. 20. 20. 20. 20.]
player alpha turn
case 2 take_open=0 discard=0
take from closed swaps with own
player alpha has now 7
open Pile has now in top 0
number of closed cards of player alpha is 8
numeric output is
[  0.   9.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 124.   9.   5.  10.  20.  20.  20.  20.  20.  20.  20.
  20.  20.  20.  20.  20.  20.  20.  -1.  20.   0.  20.  20.  20.

It works still. 

In [155]:
#level (computer mode) -1
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',-1,pile_closed)
beta=Player("beta",'computer',-1,pile_closed) 
players=[alpha,beta]
#always same player here
#all not silent and output collection
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("full numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])

player alpha turn
case 2 take_open=0 discard=0
take from closed swaps with own
player alpha has now 6
open Pile has now in top 1
number of closed cards of player alpha is 9
full numeric output is
[  0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 125.  20.  20.  20.  20.  20.  20.  20.  20.   5.  20.
  20.  12.  20.  20.  20.  20.  20.  20.  20.  -2.  20.  20.   7.  20.
   0.   0.   6.  10.  20.   0.]
player alpha numeric output is
[20. 20. 20. 20. 20. 20. 20. 20.  5. 20. 20. 12.]
player beta numeric output is
[20. 20. 20. 20. 20. 20. 20. -2. 20. 20.  7. 20.]
player alpha turn
case 2 take_open=0 discard=0
take from closed swaps with own
player alpha has now 2
open Pile has now in top 0
number of closed cards of player alpha is 8
numeric output is
[  0.   1.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 124.  20.  20.  20.  20.  20.  20.  20.  20.   5.  20.
   6.  12.  20.  20.  20.  20.  20.  20.  20.  -2.  20.  20.   7.

In [157]:
#level (computer mode) -2
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',-2,pile_closed)
beta=Player("beta",'computer',-2,pile_closed) 
players=[alpha,beta]
#always same player here
#all not silent and output collection
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("full numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])


player alpha turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 5
open Pile has now on top 3
number of closed cards of player alpha is 9
full numeric output is
[  0.   5.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 125.  20.  20.  20.  20.  20.  20.  -1.  20.   0.  20.
  20.  20.  11.  20.  20.  20.  20.  20.  -2.  20.  20.  20.  20.  20.
   0.   1.   3.   1.  20.   0.]
player alpha numeric output is
[20. 20. 20. 20. 20. 20. -1. 20.  0. 20. 20. 20.]
player beta numeric output is
[11. 20. 20. 20. 20. 20. -2. 20. 20. 20. 20. 20.]
player alpha turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 0
open Pile has now on top 7
number of closed cards of player alpha is 8
numeric output is
[  0.   3.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.
   0.   0.   0. 124.  20.   5.  20.  20.  20.  20.  -1.  20.   0.  20.
  20.  20.  11.  20.  20.  20.  20.  20.  -2.  20.  20.  20.  20.  20.
   0.   1.   7. 

In [158]:
#level (computer mode) -3
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',-3,pile_closed)
beta=Player("beta",'computer',-3,pile_closed) 
players=[alpha,beta]
#always same player here
#all not silent and output collection
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("full numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])
player,players, pile,discarded,num=turn(alpha,players,pile_closed,pile_open,False,True)
print("number of closed cards of player alpha is "+str(len(alpha.get_all_closed())))
print("numeric output is")
print(num)
print("player alpha numeric output is")
print(num[18:30])
print("player beta numeric output is")
print(num[30:42])

player alpha turn
case 0 take_open=1 discard=0
take from open swaping with own
player alpha has now 6
open Pile has now on 5
number of closed cards of player alpha is 9
full numeric output is
[  0.   6.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 125.  20.  20.  20.  20.  20.  20.  20.  20.  20.   3.
   7.  20.  20.  20.  20.  20.  20.  20.  20.   6.  20.   1.  20.  20.
   1.   0.  30.   2.  20.   0.]
player alpha numeric output is
[20. 20. 20. 20. 20. 20. 20. 20. 20.  3.  7. 20.]
player beta numeric output is
[20. 20. 20. 20. 20. 20. 20.  6. 20.  1. 20. 20.]
player alpha turn
case 0 take_open=1 discard=0
take from open swaping with own
player alpha has now 5
open Pile has now on 11
number of closed cards of player alpha is 8
numeric output is
[  0.   5.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 125.  20.  20.   6.  20.  20.  20.  20.  20.  20.   3.
   7.  20.  20.  20.  20.  20.  20.  20.  20.   6.  20.   1.  20.  20.
 

The allowed_modes function is till the same since not more are implemented now. 

In [159]:
def allowed_modes(names,nature,levels):
    #list of allowed computer level, no humman currently implemented
    comp_level_list = [0,-1,-2,-3]
    #default is true
    allowed=True
    #between 2 and 8 players and input lists of same length
    if len(names)<2 or len(names)>8 or len(names)!=len(nature) or len(names)!=len(levels):
        allowed=False
    else:
        for i in range(len(names)):
            #for computer
            if nature[i]=='computer' and any(levels[i] in comp_level_list for item in comp_level_list)==False:
                allowed=False
            #human is not implemented and will have different levels    
            #for now because human not implemented    
            if nature[i]!='computer':
                allowed=False
    return allowed             

Now I make the small changes need in the round

For changing the order of it the helper function reorder_players is used. 

In [86]:
#parameters index of acting player of a turn, and the result of the players (some kind of score)
def reorder_players(player,result):
    xx=len(result)
    array=np.zeros((xx))    
    #acting players
    array[0]=result[int(player)]
    for j in range(1,xx):
        #get scores of following players
        array[j]=result[int((j+player)%xx)] 
    return array    

In [87]:

Now I make the small changes needed in the skyjo_round function. 

[30. 40. 50. 10. 20.]


In [162]:
#parameters: player names, modes, level (all list of same length), pause per turn (seconds can be zero), 
#whether it is first round either True, or player which starts 
#silent printing optional 
#output collection optional , vanish card count is now part of numeric output
def skyjo_round(names,nature,levels,pause,first_round,silent=True,output=False):
    #check whether input is allowed and defined
    allowed=allowed_modes(names,nature,levels)
    #report and abort when not defined            
    if allowed==False:
        print("The player input \nnames:"+str(names)+"\nnature:"+str(nature)+"\nlevel:"+str(levels))
        print("is not allowed.")
    else:
        #globals (needed? all)
        global in_play, players, pile_open, pile_closed
        #create piles at the beggining
        pile_closed=Pile('create_closed',False)
        pile_open=Pile('create_open',pile_closed)
        if output==True:
            #for output collection first a list is created which is at the end converted to an numpy array
            outputs=[]
        if first_round==True:
            #In first round: create players and add the players to list 
            players=[]
            for i in range(len(names)):
                players.append(Player(names[i],nature[i],levels[i]))
        for i in range(len(names)):
            #fill Player always from pile 
            players[i].restart(pile_closed)            
        if silent==False:
            print("closed pile has "+str(len(pile_closed.list_cards))+" cards")        
        #drawing here?    does not work to be added for interactive
        #frame.set_draw_handler(draw)
        #determine start player in first round 
        if first_round==True:
            if silent==False:
                print("game starts")
            for i in range(len(players)):
                if silent==False:
                    print(players[i])
            starter=who_starts(True,players,None)
        else:
            #could omit function in that case
            starter=who_starts(False,players,first_round)
            if silent==False:
                print("new round starts")
        if silent==False: 
            print("Player "+starter.name+" starts")
        #index of starter player
        idx=players.index(starter)
        if pause>0:
            #slow round to be able to observe in GUI or print out
            time.sleep(pause)
        #first turn
        if output==False:
            turn(starter,players,pile_closed,pile_open,silent,False)
        else:
            starter,players,pile,discarded,num=turn(starter,players,pile_closed,pile_open,silent,True)
            outputs.append(num)
        if silent==False:
            print("closed pile has "+str(len(pile_closed.list_cards))+" cards")     
        #counter set to index of starter, used to get index of next player
        counter=players.index(starter)
        #play starts
        in_play=True
        #index of player who finishes first with cards
        finisher=0
        #continue until one player finishes
        while in_play==True:
            #optionally slowing down
            if pause>0:
                time.sleep(pause)
            #now alternating turns on all player while in play
            counter+=1
            player=players[counter%len(players)]
            if silent==False:
                print("next turn is of "+player.name)            
            if output==False:
                #turn with and without output collectiob
                turn(player,players,pile_closed,pile_open,silent,False)
            else:
                player,players, pile,discarded,num=turn(player,players,pile_closed,pile_open,silent,True)   
                outputs.append(num)
            if silent==False:
                print(player.name)
                print("closed pile has "+str(len(pile_closed.list_cards))+" cards") 
            #index of finishing players    
            finisher=counter%len(players)        
        #one turn of the other players at the  end
        #last player will start next round in a game of several rounds
        last_player=None
        if in_play==False:
            for i in range(len(players)-1):
                #optional slowing
                if pause>0:
                    time.sleep(pause)
                #counting to get index of players    
                counter+=1
                player=players[counter%len(players)]
                #last_player will start next round in a game
                last_player=player 
                if silent==False:
                    print("next turn is of "+player.name)
                if output==False:
                    player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open,silent,False)
                else:
                    player,players, pile,discarded,num=turn(player,players,pile_closed,pile_open,silent,True)    
                    outputs.append(num)
                if silent==False:
                    print("closed pile has "+str(len(pile_closed.list_cards))+" cards")
        if silent==False:
            print(str(counter-idx+1)+" turns were made")            
  
        #get score of round, list        
        scores=[]
        for i in range(len(players)):
            #get score of each player
            scores.append(players[i].get_score())   
        #need to consider who finished the round
        #if not the unique lowest (and positive) finishs first then score of the finisher is doubled
        #not_finisher controll parameter to avoid several doublings
        not_finisher=False
        if scores[finisher]>0:
            for i in range(len(players)):       
                if i!=finisher and not_finisher==False:
                    if scores[finisher]>=scores[i]:
                        scores[finisher]*=2
                        not_finisher=True
        if silent==False:
            print("score of round is "+str(scores))
            if output==True:
                print(str(vanish_count)+" times equal card vanished")
        if output==True: 
            #output mostly the output of the round, plus round number and 
            #at the end the scores in the same order as players in that turn
            num2=np.zeros((len(num)+len(scores)+1,len(outputs)),int)
            for i in range(len(outputs)):
                num2[:len(num),i]=outputs[i]
                #turn number
                num2[len(num),i]=i
                #reorder scores
                res=reorder_players(num2[0,i],scores)
                #pass to array
                num2[len(num)+1:len(num)+len(scores)+1,i]=res
        if output==False:    
            return scores, counter-idx+1, last_player
        else:    
            return scores, counter-idx+1, last_player, num2    

First I test a round with print and numerical output

In [166]:
#creates names and properties of player
names=['alpha','beta','gamma']
nature=['computer','computer','computer']
levels=[0,0,0]
start_time=time.time()
scores,turns,last_player,num=skyjo_round(names,nature,levels,0,True,False,True)
stop_time=time.time()
print(str(turns)+" turns were made")
print("score of the round is "+str(scores))
print(str(sum(num[len(num)-5,:]))+" times card vanished")
print(num.shape)
print("indixes of players")
print(num[0,:].T)
print("scores of them")
print(num[len(num)-3:len(num),:].T)
print("the round needed "+str(stop_time-start_time)+" seconds")

closed pile has 113 cards
game starts
Player alpha is a computer in level 0.
Player beta is a computer in level 0.
Player gamma is a computer in level 0.
Player alpha starts
player alpha turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 3
open Pile has now on top 5
closed pile has 112 cards
next turn is of beta
player beta turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 8
open Pile has now on top 4
beta
closed pile has 111 cards
next turn is of gamma
player gamma turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 7
open Pile has now on top 1
gamma
closed pile has 110 cards
next turn is of alpha
player alpha turn
case 0 take_open=1 discard=0
take from open swaping with own
player alpha has now 1
open Pile has now on 11
alpha
closed pile has 110 cards
next turn is of beta
player beta turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 6
open Pile has now on top 5
beta
closed pile has 10

Now without numerical output. 

In [168]:
#creates names and properties of player
names=['alpha','beta','gamma']
nature=['computer','computer','computer']
levels=[0,0,0]
start_time=time.time()
scores,turns,last_player=skyjo_round(names,nature,levels,0,True,False,False)
stop_time=time.time()
print(str(turns)+" turns were made")
print("score of the round is "+str(scores))
#creates names and properties of player
names=['alpha','beta','gamma']
nature=['computer','computer','computer']
levels=[0,0,0]
start_time=time.time()
scores,turns,last_player,num=skyjo_round(names,nature,levels,0,True,False,True)
stop_time=time.time()
print(str(turns)+" turns were made")
print("score of the round is "+str(scores))
print(str(sum(num[len(num)-5,:]))+" times card vanished")
print("indixes of players")
print(num[0,:].T)
print("scores of them")
print(num[len(num)-3:len(num),:].T)
#counter now fine, refill now also implemented (rarely needed for 2 player but will be important for more)
print("the round needed "+str(stop_time-start_time)+" seconds") #it got slower somehow #it got slower somehow is about twice as fast

closed pile has 113 cards
game starts
Player alpha is a computer in level 0.
Player beta is a computer in level 0.
Player gamma is a computer in level 0.
Player gamma starts
player gamma turn
case 2 take_open=0 discard=0
take from closed swaps with own
player gamma has now -2
open Pile has now in top 2
closed pile has 112 cards
next turn is of alpha
player alpha turn
case 0 take_open=1 discard=0
take from open swaping with own
player alpha has now 2
open Pile has now on 0
alpha
closed pile has 112 cards
next turn is of beta
player beta turn
case 0 take_open=1 discard=0
take from open swaping with own
player beta has now 0
open Pile has now on 4
beta
closed pile has 112 cards
next turn is of gamma
player gamma turn
case 1 take_open=0 discard=1
take from closed discards it
Player opens 9
open Pile has now on top 7
gamma
closed pile has 111 cards
next turn is of alpha
player alpha turn
case 0 take_open=1 discard=0
take from open swaping with own
player alpha has now 7
open Pile has now on

Now many rounds silent to see whether it terminates sometimes. 

In [169]:
#testing for termination
names=['alpha','beta','gamma','delta']
nature=['computer','computer','computer','computer']
levels=[0,0,0,0]
nn=int(1000)
result=np.zeros((len(names)+2,nn))
start_time=time.time()
for i in range(nn):
    print(f"run number {i}")
    scores,turns,last_player=skyjo_round(names,nature,levels,0,True,True,False) 
    for j in range(len(names)):
        result[j,i]=scores[j]
    result[len(names),i]=turns
stop_time=time.time()
print("a round needed "+str((stop_time-start_time)/nn)+" seconds")
print("minimum, mean, maximum number of turns")
print(min(result[len(names)]),np.mean(result[len(names)]),max(result[len(names)]))

run number 0
run number 1
run number 2
run number 3
run number 4
run number 5
run number 6
run number 7
run number 8
run number 9
run number 10
run number 11
run number 12
run number 13
run number 14
run number 15
run number 16
run number 17
run number 18
run number 19
run number 20
run number 21
run number 22
run number 23
run number 24
run number 25
run number 26
run number 27
run number 28
run number 29
run number 30
run number 31
run number 32
run number 33
run number 34
run number 35
run number 36
run number 37
run number 38
run number 39
run number 40
run number 41
run number 42
run number 43
run number 44
run number 45
run number 46
run number 47
run number 48
run number 49
run number 50
run number 51
run number 52
run number 53
run number 54
run number 55
run number 56
run number 57
run number 58
run number 59
run number 60
run number 61
run number 62
run number 63
run number 64
run number 65
run number 66
run number 67
run number 68
run number 69
run number 70
run number 71
ru

run number 610
run number 611
run number 612
run number 613
run number 614
run number 615
run number 616
run number 617
run number 618
run number 619
run number 620
run number 621
run number 622
run number 623
run number 624
run number 625
run number 626
run number 627
run number 628
run number 629
run number 630
run number 631
run number 632
run number 633
run number 634
run number 635
run number 636
run number 637
run number 638
run number 639
run number 640
run number 641
run number 642
run number 643
run number 644
run number 645
run number 646
run number 647
run number 648
run number 649
run number 650
run number 651
run number 652
run number 653
run number 654
run number 655
run number 656
run number 657
run number 658
run number 659
run number 660
run number 661
run number 662
run number 663
run number 664
run number 665
run number 666
run number 667
run number 668
run number 669
run number 670
run number 671
run number 672
run number 673
run number 674
run number 675
run number

Now many rounds with output. 

In [170]:
names=['alpha','beta','gamma','delta']
nature=['computer','computer','computer','computer']
levels=[0,0,0,0]
nn=int(10000)
result=np.zeros((len(names)+2,nn))
start_time=time.time()
for i in range(nn):
    print(f"run number {i}")
    scores,turns,last_player,numeric=skyjo_round(names,nature,levels,0,True,True,True) 
    for j in range(len(names)):
        result[j,i]=scores[j]
    result[len(names),i]=turns
    result[len(names)+1,i]=sum(numeric[len(numeric)-6,:])
stop_time=time.time()
print("a round needed "+str((stop_time-start_time)/nn)+" seconds")
print("minimum, mean, maximum number of turns")
print(min(result[len(names)]),np.mean(result[len(names)]),max(result[len(names)]))
print("minimum, mean, maximum number and total number of vanishing cards events")
print(min(result[len(names)+1]),np.mean(result[len(names)+1]),max(result[len(names)+1]),sum(result[len(names)+1]))

run number 0
run number 1
run number 2
run number 3
run number 4
run number 5
run number 6
run number 7
run number 8
run number 9
run number 10
run number 11
run number 12
run number 13
run number 14
run number 15
run number 16
run number 17
run number 18
run number 19
run number 20
run number 21
run number 22
run number 23
run number 24
run number 25
run number 26
run number 27
run number 28
run number 29
run number 30
run number 31
run number 32
run number 33
run number 34
run number 35
run number 36
run number 37
run number 38
run number 39
run number 40
run number 41
run number 42
run number 43
run number 44
run number 45
run number 46
run number 47
run number 48
run number 49
run number 50
run number 51
run number 52
run number 53
run number 54
run number 55
run number 56
run number 57
run number 58
run number 59
run number 60
run number 61
run number 62
run number 63
run number 64
run number 65
run number 66
run number 67
run number 68
run number 69
run number 70
run number 71
ru

run number 587
run number 588
run number 589
run number 590
run number 591
run number 592
run number 593
run number 594
run number 595
run number 596
run number 597
run number 598
run number 599
run number 600
run number 601
run number 602
run number 603
run number 604
run number 605
run number 606
run number 607
run number 608
run number 609
run number 610
run number 611
run number 612
run number 613
run number 614
run number 615
run number 616
run number 617
run number 618
run number 619
run number 620
run number 621
run number 622
run number 623
run number 624
run number 625
run number 626
run number 627
run number 628
run number 629
run number 630
run number 631
run number 632
run number 633
run number 634
run number 635
run number 636
run number 637
run number 638
run number 639
run number 640
run number 641
run number 642
run number 643
run number 644
run number 645
run number 646
run number 647
run number 648
run number 649
run number 650
run number 651
run number 652
run number

run number 1143
run number 1144
run number 1145
run number 1146
run number 1147
run number 1148
run number 1149
run number 1150
run number 1151
run number 1152
run number 1153
run number 1154
run number 1155
run number 1156
run number 1157
run number 1158
run number 1159
run number 1160
run number 1161
run number 1162
run number 1163
run number 1164
run number 1165
run number 1166
run number 1167
run number 1168
run number 1169
run number 1170
run number 1171
run number 1172
run number 1173
run number 1174
run number 1175
run number 1176
run number 1177
run number 1178
run number 1179
run number 1180
run number 1181
run number 1182
run number 1183
run number 1184
run number 1185
run number 1186
run number 1187
run number 1188
run number 1189
run number 1190
run number 1191
run number 1192
run number 1193
run number 1194
run number 1195
run number 1196
run number 1197
run number 1198
run number 1199
run number 1200
run number 1201
run number 1202
run number 1203
run number 1204
run numb

run number 1688
run number 1689
run number 1690
run number 1691
run number 1692
run number 1693
run number 1694
run number 1695
run number 1696
run number 1697
run number 1698
run number 1699
run number 1700
run number 1701
run number 1702
run number 1703
run number 1704
run number 1705
run number 1706
run number 1707
run number 1708
run number 1709
run number 1710
run number 1711
run number 1712
run number 1713
run number 1714
run number 1715
run number 1716
run number 1717
run number 1718
run number 1719
run number 1720
run number 1721
run number 1722
run number 1723
run number 1724
run number 1725
run number 1726
run number 1727
run number 1728
run number 1729
run number 1730
run number 1731
run number 1732
run number 1733
run number 1734
run number 1735
run number 1736
run number 1737
run number 1738
run number 1739
run number 1740
run number 1741
run number 1742
run number 1743
run number 1744
run number 1745
run number 1746
run number 1747
run number 1748
run number 1749
run numb

run number 2201
run number 2202
run number 2203
run number 2204
run number 2205
run number 2206
run number 2207
run number 2208
run number 2209
run number 2210
run number 2211
run number 2212
run number 2213
run number 2214
run number 2215
run number 2216
run number 2217
run number 2218
run number 2219
run number 2220
run number 2221
run number 2222
run number 2223
run number 2224
run number 2225
run number 2226
run number 2227
run number 2228
run number 2229
run number 2230
run number 2231
run number 2232
run number 2233
run number 2234
run number 2235
run number 2236
run number 2237
run number 2238
run number 2239
run number 2240
run number 2241
run number 2242
run number 2243
run number 2244
run number 2245
run number 2246
run number 2247
run number 2248
run number 2249
run number 2250
run number 2251
run number 2252
run number 2253
run number 2254
run number 2255
run number 2256
run number 2257
run number 2258
run number 2259
run number 2260
run number 2261
run number 2262
run numb

...

run number 7619
run number 7620
run number 7621
run number 7622
run number 7623
run number 7624
run number 7625
run number 7626
run number 7627
run number 7628
run number 7629
run number 7630
run number 7631
run number 7632
run number 7633
run number 7634
run number 7635
run number 7636
run number 7637
run number 7638
run number 7639
run number 7640
run number 7641
run number 7642
run number 7643
run number 7644
run number 7645
run number 7646
run number 7647
run number 7648
run number 7649
run number 7650
run number 7651
run number 7652
run number 7653
run number 7654
run number 7655
run number 7656
run number 7657
run number 7658
run number 7659
run number 7660
run number 7661
run number 7662
run number 7663
run number 7664
run number 7665
run number 7666
run number 7667
run number 7668
run number 7669
run number 7670
run number 7671
run number 7672
run number 7673
run number 7674
run number 7675
run number 7676
run number 7677
run number 7678
run number 7679
run number 7680
run numb

run number 8159
run number 8160
run number 8161
run number 8162
run number 8163
run number 8164
run number 8165
run number 8166
run number 8167
run number 8168
run number 8169
run number 8170
run number 8171
run number 8172
run number 8173
run number 8174
run number 8175
run number 8176
run number 8177
run number 8178
run number 8179
run number 8180
run number 8181
run number 8182
run number 8183
run number 8184
run number 8185
run number 8186
run number 8187
run number 8188
run number 8189
run number 8190
run number 8191
run number 8192
run number 8193
run number 8194
run number 8195
run number 8196
run number 8197
run number 8198
run number 8199
run number 8200
run number 8201
run number 8202
run number 8203
run number 8204
run number 8205
run number 8206
run number 8207
run number 8208
run number 8209
run number 8210
run number 8211
run number 8212
run number 8213
run number 8214
run number 8215
run number 8216
run number 8217
run number 8218
run number 8219
run number 8220
run numb

run number 8691
run number 8692
run number 8693
run number 8694
run number 8695
run number 8696
run number 8697
run number 8698
run number 8699
run number 8700
run number 8701
run number 8702
run number 8703
run number 8704
run number 8705
run number 8706
run number 8707
run number 8708
run number 8709
run number 8710
run number 8711
run number 8712
run number 8713
run number 8714
run number 8715
run number 8716
run number 8717
run number 8718
run number 8719
run number 8720
run number 8721
run number 8722
run number 8723
run number 8724
run number 8725
run number 8726
run number 8727
run number 8728
run number 8729
run number 8730
run number 8731
run number 8732
run number 8733
run number 8734
run number 8735
run number 8736
run number 8737
run number 8738
run number 8739
run number 8740
run number 8741
run number 8742
run number 8743
run number 8744
run number 8745
run number 8746
run number 8747
run number 8748
run number 8749
run number 8750
run number 8751
run number 8752
run numb

run number 9215
run number 9216
run number 9217
run number 9218
run number 9219
run number 9220
run number 9221
run number 9222
run number 9223
run number 9224
run number 9225
run number 9226
run number 9227
run number 9228
run number 9229
run number 9230
run number 9231
run number 9232
run number 9233
run number 9234
run number 9235
run number 9236
run number 9237
run number 9238
run number 9239
run number 9240
run number 9241
run number 9242
run number 9243
run number 9244
run number 9245
run number 9246
run number 9247
run number 9248
run number 9249
run number 9250
run number 9251
run number 9252
run number 9253
run number 9254
run number 9255
run number 9256
run number 9257
run number 9258
run number 9259
run number 9260
run number 9261
run number 9262
run number 9263
run number 9264
run number 9265
run number 9266
run number 9267
run number 9268
run number 9269
run number 9270
run number 9271
run number 9272
run number 9273
run number 9274
run number 9275
run number 9276
run numb

run number 9739
run number 9740
run number 9741
run number 9742
run number 9743
run number 9744
run number 9745
run number 9746
run number 9747
run number 9748
run number 9749
run number 9750
run number 9751
run number 9752
run number 9753
run number 9754
run number 9755
run number 9756
run number 9757
run number 9758
run number 9759
run number 9760
run number 9761
run number 9762
run number 9763
run number 9764
run number 9765
run number 9766
run number 9767
run number 9768
run number 9769
run number 9770
run number 9771
run number 9772
run number 9773
run number 9774
run number 9775
run number 9776
run number 9777
run number 9778
run number 9779
run number 9780
run number 9781
run number 9782
run number 9783
run number 9784
run number 9785
run number 9786
run number 9787
run number 9788
run number 9789
run number 9790
run number 9791
run number 9792
run number 9793
run number 9794
run number 9795
run number 9796
run number 9797
run number 9798
run number 9799
run number 9800
run numb

A function takes longer with output and  even more with prints. Still, for 4 players of computer level 0 it only takes 3.9 (with revious version 3.4) millisceond per round, not much and the clarity improved. 
A vanishing event happens about 0.077 times per round for 4 players of computer level 0. 

Now I check the gmae function, again a few changes are needed because integration of the vanishing card count into the numeric output. 

In [171]:
#parameters: player names, modes, level (all list of same length),
#pause length (can be zero) 
#printing optional 
#output collection optional 
def skyjo_game(names,nature,levels,pause,silent=True,output=False):
    #check whether parameters are allowed and defined)
    allowed=allowed_modes(names,nature,levels)
    #report and abort when not defined            
    if allowed==False:
        print("The player input \nnames:"+str(names)+"\nnature:"+str(nature)+"\nlevel:"+str(levels))
        print("is not allowed.")
    else:
        #play the game
        #total book keeping round and score (list of player length)
        round_counter=0
        tot_score=[]
        #starts at zero
        for i in range(len(names)):
            tot_score.append(0)
        #marker for game over   
        over=False
        #create players and add to list
        players=[]
        #for output collection first a list which is at the end converted to array
        for i in range(len(names)):
            #player nature creation
            players.append(Player(names[i],nature[i],levels[i]))
        if output==False:
            score,turns,last_player=skyjo_round(names,nature,levels,pause,True,True,output) 
        else:
            #for output collection first a list which is at the end converted to array
            outputs=[]
            score,turns,last_player,num=skyjo_round(names,nature,levels,pause,True,True,output)   
            outputs.append(num)
        if silent==False:
            print("New Skyjo game")
            print("first round")
            print("score is "+str(score))
        
        for i in range(len(tot_score)):
            tot_score[i]+=score[i]
        if max(tot_score)>=100:
            over=True
        else:
            #more rounds until first player has at least 100
            while max(tot_score)<100:
                round_counter+=1
                if silent==False:
                    print("playing round "+str(round_counter+1))
                if output==False:
                    score,turns,last_player=skyjo_round(names,nature,levels,pause,last_player,True,output)
                else: 
                    score,turns,last_player,num=skyjo_round(names,nature,levels,pause,last_player,True,output)
                    outputs.append(num)
                for i in range(len(tot_score)):
                    tot_score[i]+=score[i] 
                if silent==False:
                    print("current score is "+str(tot_score))   
            else:
                over=True
        if over==True:
            if silent==False:
                print("game is over")   
            winner=[]    
            for i in range(len(players)):
                if tot_score[i]==min(tot_score):
                    winner.append(1)
                    if silent==False:
                        print("Player "+players[i].name+" won")    
                else:
                    winner.append(0)
            if output==True:
                #combine outputs in one array 
                #figure out size of it 
                row_counter=0
                for i in range(round_counter+1):
                    row_counter+=outputs[i].shape[1]
                #final numeric output
                #include besides propagate of round also round number, and reordered to player winner vector 
                final=np.zeros((outputs[0].shape[0]+1+len(players),row_counter),int)
                row_counter=0
                for i in range(round_counter+1):
                    final[0:outputs[0].shape[0],row_counter:row_counter+outputs[i].shape[1]]=outputs[i]
                    for j in range(outputs[i].shape[1]):
                        final[outputs[0].shape[0],row_counter+j]=i
                        #reorder to the winner to the player order
                        final[outputs[0].shape[0]+1:outputs[0].shape[0]+len(players)+1,row_counter+j]=reorder_players(final[0,row_counter+j],winner)
                    row_counter+=outputs[i].shape[1] 
                return final, winner
            else:
                #no numeric, output just winner
                return winner

Now I test the function with print and with less output. 

In [193]:
names=['alpha','beta']
nature=['computer','computer']
levels=[-3,-1]
print(names,nature,levels)
winner=skyjo_game(names,nature,levels,0,False,False)

['alpha', 'beta'] ['computer', 'computer'] [-3, -1]
New Skyjo game
first round
score is [47, 66]
playing round 2
current score is [106, 135]
game is over
Player alpha won


Now with numeric output.

In [195]:
names=['alpha','beta','gamma']
nature=['computer','computer','computer']
levels=[0,0,0]
print(names,nature,levels)
numeric,winner=skyjo_game(names,nature,levels,0,False,True)
print("winner vector")
print(winner)
print ("number of vanishing cards events")
print(sum(numeric[numeric.shape[0]-9,:]))
print("shape of numeric array")
print(numeric.shape)
print("acting player, score of round, round number, winner vector")
for i in range(numeric.shape[1]):
    print(numeric[0,i],numeric[numeric.shape[0]-7:numeric.shape[0],i])

['alpha', 'beta', 'gamma'] ['computer', 'computer', 'computer'] [0, 0, 0]
New Skyjo game
first round
score is [62, 48, 63]
playing round 2
current score is [107, 156, 127]
game is over
Player alpha won
winner vector
[1, 0, 0]
number of vanishing cards events
0
shape of numeric array
(68, 93)
acting player, score of round, round number, winner vector
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0]
1 [48 63 62  0  0  0  1]
2 [63 62 48  0  0  1  0]
0 [62 48 63  0  1  0  0

The game function and its output works.

Below I build the function to draw the results in the GUI  screen. It is not yet fully working, it is not needed for tarining the computer. It will further developed elsewhere.

In [ ]:
# draw function
def draw(canvas):
    global pile_open,pile_closed, players
    #display the top most card
    p_open=pile_open.list_cards[-1]
    p_closed=pile_closed.list_cards[-1]
    # test to make sure that card.draw works
    p_open.draw(canvas)
    p_closed.draw(canvas)
    for i in range(len(players)):
        for j in range(12):
            #only cards which exist are drawn:
            if players[i].exist[j]==1:
                card=players[i].list_cards[j]
                if len(players)==2:
                    drawpos=list(card.position)
                    drawpos[0]=i*290+players[i].positionx[j]
                    drawpos[1]=100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-3.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%2)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//2)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-5.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%3)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//3)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%3)*290,180*(1+(i//3))+90),15,'Black')     
                if abs(len(players)-7.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%4)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//4)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%4)*290,180*(1+(i//4))+90),15,'Black')    

Drawing works now for all allowed player numbers. 

In [ ]:
def hit_start():
    global start
    #better but still not really
    players,scores,turns=skyjo_round(names,nature,levels,1) 
    #only appears at the end

In [ ]:
#seems not work well in the notebook sometimes, not clear whether real problem
#likely ignored since notebook is not so important here
names=['alpha','beta']
nature=['computer','computer']
levels=[0,0]
if len(names)==2:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50+30) 
if abs(len(names)-3.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50*2+60)
if abs(len(names)-5.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(2)+10, 100+3*50*2+60)    
if abs(len(names)-7.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(3)+20, 100+3*50*2+60)  
frame.set_canvas_background("White")
frame.add_button("start", hit_start, 200)    
frame.set_draw_handler(draw)
frame.start()
players,scores,turns=skyjo_round(names,nature,levels,1) 
#drawing of comuter does not work 

In [ ]:
exit()
